In [2]:
#!pip install psycopg2


  Using cached psycopg2-2.9.10-cp312-cp312-linux_x86_64.whl


In [3]:
import psycopg2
import pandas as pd
import csv

### verify the conexion
try:
    # Connect to the database
    conn = psycopg2.connect(
        host="pg.ujueta.com",
        port="5432",
        user="consultoria",
        password="LmKTXJBXya!14]f9!2k]",
        database="postgres",
    )
except psycopg2.Error as e:
    print("Error connecting to the database:")
    print(e)
else:
    print("Connection established successfully")

Connection established successfully


In [4]:

### ventas diario apiladas
def export_forecast_to_csv():
    try:
        # connect to the database
        conn = psycopg2.connect(
            host="pg.ujueta.com",
            port="5432",
            user="consultoria",
            password="LmKTXJBXya!14]f9!2k]",
            database="postgres",
        )

        # create cursor
        with conn.cursor() as cur:

            # ececute a sql query

            cur.execute("SELECT * FROM forecast.ventas_diario")

            # fetch the results
            results = cur.fetchall()

            # open a file in the downloads folder

            with open(
                "/home/usuario/Escritorio/Consultorias_Empresariales/Ujueta/Datos/outputs.csv",
                "w",
                newline="",
            ) as f:
                # create a csv writer
                writer = csv.writer(f)

                # write the column names
                writer.writerow([col[0] for col in cur.description])

                # write the query results
                writer.writerows(results)
    except Exception as e:
        print(f"An error ocurred: {e}")
    finally:
        # close the cursor and connection
        if conn:
            conn.close()


# Call the function to export data to csv
export_forecast_to_csv()


In [8]:


### manipulate data

import pandas as pd

# carga de datas
data = pd.read_csv(
    "/home/usuario/Escritorio/Consultorias_Empresariales/Ujueta/Datos/outputs.csv"
)
# convertir en dataframe
df = pd.DataFrame(data)
df.head()

,docdate,codigo_articulo,cantidad
0,2022-01-03,AUACALR4,1
1,2022-01-03,AUACCH1,5
2,2022-01-03,AUACPB400,1
3,2022-01-03,AUACRIM4F,5
4,2022-01-03,AUACSH1000,3


In [17]:
df.shape

(87005, 3)

In [15]:
# List of SKUs to filter
selected_skus = [
    'AUACAE30', 'AUACPB400', 'AUACRIM4F', 'AUACSH1000', 'HEELAG1141',
    'HEELAG1142KIT', 'HEELPW1770', 'HEELXID20', 'HEFUFCD12KIT',
    'HEFUFCD21', 'HEFUFD52', 'HEFUFG71', 'SOELCSVM501', 'SOELCSVM530',
    'SOELSI6140DV', 'SOFUFW181', 'SOFUFW185', 'SOFUFW33', 'SOFUFW35',
    'SOSWP2-517'
]

# Filter the dataset
filtered_df = df[df['codigo_articulo'].isin(selected_skus)]

In [16]:
print(filtered_df)

          docdate codigo_articulo  cantidad
2      2022-01-03       AUACPB400         1
3      2022-01-03       AUACRIM4F         5
4      2022-01-03      AUACSH1000         3
10     2022-01-03      HEELAG1141        56
22     2022-01-03      HEELPW1770         4
...           ...             ...       ...
86951  2024-11-13    SOELSI6140DV        88
86965  2024-11-13       SOFUFW181         9
86969  2024-11-13        SOFUFW33         9
86970  2024-11-13        SOFUFW35         4
87001  2024-11-13      SOSWP2-517         4

[13060 rows x 3 columns]


In [18]:
data_highdda = filtered_df[filtered_df['docdate'] >= '2023-01-01']
data_highdda

,docdate,codigo_articulo,cantidad
26811,2023-01-02,HEELAG1141,5
26827,2023-01-02,HEFUFCD12KIT,212
26828,2023-01-02,HEFUFCD21,4
26830,2023-01-02,HEFUFD52,14
26831,2023-01-02,HEFUFG71,1
...,...,...,...
86951,2024-11-13,SOELSI6140DV,88
86965,2024-11-13,SOFUFW181,9
86969,2024-11-13,SOFUFW33,9
86970,2024-11-13,SOFUFW35,4


In [34]:
#========================================================================================================================================
# Ajuste de Datos para statsforecast
#========================================================================================================================================

#-- Dato de ejemplo para función

def data_sf(df, date = 'docdate', y = 'cantidad', unique_id = 'codigo_articulo'):
    df_m = df[[date, y, unique_id]]
    df_m.rename(columns={date : 'ds', y :'y', unique_id: 'unique_id'}, inplace=True)
  
    return df_m

df_m = data_sf(data_highdda, date = 'docdate', y = 'cantidad', unique_id = 'codigo_articulo')
df_m
df_m.info()




<class 'pandas.core.frame.DataFrame'>
Index: 8739 entries, 26811 to 87001
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ds         8739 non-null   object
 1   y          8739 non-null   int64 
 2   unique_id  8739 non-null   object
dtypes: int64(1), object(2)
memory usage: 273.1+ KB


In [35]:
#========================================================================================================================================
# Colección de modelos
#========================================================================================================================================
from statsforecast import StatsForecast #- Para instanciar los  modelos

from statsforecast.models import (
    AutoARIMA,
    AutoETS,
    HoltWinters,
    ADIDA,
    CrostonClassic as Croston, 
    IMAPA,
    TSB,
    HistoricAverage,
    DynamicOptimizedTheta as DOT,
    SeasonalNaive
)

#-- Parametros

# Número de días en el futuro a pronosticar
horizon = 30
# Ventana estacional: es 7 porque tenemos datos diarios
season_length = 7
# El número de dias que el modelo usará para hacer el forecast 
window_size = 6*30

# Lista de modelos a evaluar
models = [
    AutoARIMA(season_length=season_length),
    AutoETS(season_length=season_length),
    HoltWinters(season_length=season_length),
    ADIDA(),
    Croston(),
    IMAPA(),
    TSB(alpha_d = 0.2, alpha_p = 0.2),
    SeasonalNaive(season_length=season_length),
    HistoricAverage(),
    DOT(season_length=season_length)
]

In [36]:
# StatsForecast.plot(df_m)

# Instanciando StatsForecast como sf
sf = StatsForecast( 
    models=models,
    freq='D', 
    n_jobs=-1,
)


In [37]:
# Cross Validation
from functools import partial
from utilsforecast.evaluation import evaluate
from utilsforecast.losses import mape, mase, mse, smape

crossvaldation_df = sf.cross_validation(
    df=df_m,
    h=horizon,
    step_size=horizon,
    n_windows=3
)



/home/usuario/myenv/lib/python3.12/site-packages/statsforecast/core.py:494: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


,ds,cutoff,y,AutoARIMA,AutoETS,HoltWinters,ADIDA,CrostonClassic,IMAPA,TSB,SeasonalNaive,HistoricAverage,DynamicOptimizedTheta
unique_id,,,,,,,,,,,,,
AUACAE30,2024-06-26,2024-06-25,1.0,9.327861,9.616291,9.521786,8.798204,8.79845,8.798204,7.686816,1.0,6.916923,8.798204
AUACAE30,2024-06-27,2024-06-25,7.0,7.402890,9.630873,9.143023,8.798204,8.79845,8.798204,7.686816,21.0,6.916923,8.798204
AUACAE30,2024-06-28,2024-06-25,1.0,7.210393,9.645454,8.901863,8.798204,8.79845,8.798204,7.686816,23.0,6.916923,8.798204
AUACAE30,2024-06-29,2024-06-25,2.0,9.327861,9.660037,9.114636,8.798204,8.79845,8.798204,7.686816,1.0,6.916923,8.798204
AUACAE30,2024-07-02,2024-06-25,2.0,8.750370,9.674619,9.494345,8.798204,8.79845,8.798204,7.686816,7.0,6.916923,8.798204


In [40]:
def evaluate_cross_validation(df, metric):
    models = df.drop(columns=['unique_id', 'ds', 'cutoff', 'y'], errors='ignore').columns.tolist()
    evals = []
    # Calculate loss for every unique_id and cutoff.    
    for cutoff in df['cutoff'].unique():
        eval_ = evaluate(df[df['cutoff'] == cutoff], metrics=[metric], models=models)
        evals.append(eval_)
    evals = pd.concat(evals)
    evals = evals.groupby('unique_id').mean(numeric_only=True) # Averages the error metrics for all cutoffs for every combination of model and unique_id
    evals['best_model'] = evals.idxmin(axis=1)
    return evals

evaluation_df = evaluate_cross_validation(crossvaldation_df.reset_index(drop=False), smape)
evaluation_df.head(22)

,AutoARIMA,AutoETS,HoltWinters,ADIDA,CrostonClassic,IMAPA,TSB,SeasonalNaive,HistoricAverage,DynamicOptimizedTheta,best_model
unique_id,,,,,,,,,,,
AUACAE30,0.498531,0.512610,0.509481,0.486631,0.486626,0.486631,0.465620,0.495371,0.465732,0.486631,TSB
AUACPB400,0.766709,0.392799,0.393811,0.396940,0.397093,0.396940,0.389711,0.482691,0.401259,0.396940,TSB
AUACRIM4F,0.591399,0.365819,0.369001,0.376410,0.376411,0.376410,0.381740,0.443935,0.365817,0.377436,HistoricAverage
AUACSH1000,0.496414,0.458045,0.441472,0.497180,0.469138,0.497180,0.480322,0.590622,0.465529,0.470731,HoltWinters
HEELAG1141,0.738691,0.624233,0.634264,0.608757,0.606525,0.608757,0.549109,0.545355,0.609268,0.608757,SeasonalNaive
HEELAG1142KIT,0.625081,0.604881,0.639477,0.605192,0.605415,0.605192,0.581615,0.557173,0.558834,0.605192,SeasonalNaive
HEELPW1770,0.538930,0.593584,0.584314,0.651698,0.654482,0.651698,0.674301,0.560684,0.593385,0.651698,AutoARIMA
HEELXID20,0.459891,0.383898,0.383450,0.388551,0.390314,0.388551,0.390070,0.399285,0.379783,0.391438,HistoricAverage
HEFUFCD12KIT,0.634410,0.466394,0.470606,0.502171,0.482715,0.502171,0.494058,0.683165,0.466399,0.483779,AutoETS


In [ ]:
#--- Selección del mejor modelo
fcst_df = sf.forecast(df=df_m, 
                      h=horizon, 
                      ## level=[90]
                     )
fcst_df.head()

In [ ]:
sf.plot(df_m,fcst_df)

In [ ]:
def get_best_model_forecast(forecasts_df, evaluation_df):
    col_bm = evaluation_df['best_model'].values[0]
    df = forecasts_df[col_bm] # Wide to long 
    df = df.reset_index(drop=False)
    df.columns = ['unique_id', 'best_model']
    return df

prod_forecasts_df = get_best_model_forecast(forecasts_df = fcst_df, 
                                            evaluation_df = evaluation_df)
prod_forecasts_df.head()